This example will perform character recognition through user input into the touch screen of an STM3240G-Evaluation board using the MNIST dataset.

In [1]:
#%tensorflow_version 2.x
#!apt-get install -y xxd

! pip uninstall -y tensorflow
#! pip install -q tf-nightly
#! pip install -q tensorflow-model-optimization

import tensorflow as tf
from tensorflow import keras

(images_train, labels_train), (images_test, labels_test) = tf.keras.datasets.mnist.load_data()

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


Importing Tensorflow allows you to use its API to load the MNIST dataset. It should be noted that we need to use TF version <1.14 as this version includes the fully connected operation version 3 which is incompatible with the micro interpreters version 2.

In [2]:
import matplotlib.pyplot as plt
test_index = 12345
print(labels_train[test_index])
plt.imshow(images_train[test_index], cmap='Greys')

3


The input of the neural network needs to know the input shape that it is going to be fed

In [3]:
input_shape = images_train.shape
print("{} images, each with shape of {} pixels x {} pixels".format(input_shape[0], input_shape[1], input_shape[2]))

60000 images, each with shape of 28 pixels x 28 pixels


The input shape for the model must be reshaped to 4D as the current shape does not show that each pixel is a 1D array where only the greyscale value (0-255) is stored. The input tensor's shape will be 3D as it will take a single-channel image. 

In [4]:
images_train = images_train.reshape(images_train.shape[0], images_train.shape[1], images_train.shape[2], 1)
images_test = images_test.reshape(images_test.shape[0], images_test.shape[1], images_test.shape[2], 1)
input_tensor_shape = (images_test.shape[1], images_test.shape[2], 1)
print("Input shape: {}".format(input_shape))

Input shape: (60000, 28, 28)


The greyscale values stores in the images' pixels are 8 bit values and need to be normalized into floats between 0-1.

In [5]:
images_train = images_train.astype('float32')
images_test = images_test.astype('float32')
images_train /= 255
images_test /= 255

In [35]:
#Counter initialization for iteration count
count=0

Random Number Generator for generating random combination of input parameters 

In [1]:
#Random Number generator 
import random
Model_Configuration= {
        
    'layer1': {
    'Iteration':str(count),
    'layer1_name': 'Conv2D',
    'param1':  str(random.choice(range(2, 50, 2))),
    'param2':  str(random.choice(range(1, 6, 3))),
    'param3': '3',
    }, 
    'layer2': {
    'Iteration':str(count),
    'layer2_name': 'Maxpool',
    'param1':  str(random.choice(range(2, 10, 2))),
    'param2':  str(random.choice(range(2, 10, 2))),
    }
    }


NameError: name 'count' is not defined

In [84]:
#JSON Module_Configurator being populated without Appending
# data = json.dumps(Model_Configuration)
# with open("/Users/priyadalal/Desktop/Append_Configuration.json","w") as f:
#     f.write(data)
    

Input configuration JSON file is used to store list of different combination of Input parameters.

In [85]:
#Appending procedure JSON
import os
import random

def append_json():
    a = []
    if not os.path.isfile("/Users/priyadalal/Desktop/InputParamComb.json"):
        a.append(Model_Configuration)
        with open("/Users/priyadalal/Desktop/InputParamComb.json", mode='w') as f:
            f.write(json.dumps(a, indent=2))
    else:
        with open("/Users/priyadalal/Desktop/InputParamComb.json") as feedsjson:
            feeds = json.load(feedsjson)

        feeds.append(Model_Configuration)
        with open("/Users/priyadalal/Desktop/InputParamComb.json", mode='w') as f:
            f.write(json.dumps(feeds, indent=2))
                



Loading the parameters from JSON file as input to our Model.

In [86]:
# # get model configuration params from our json file which is being appended
import json 

append_json()

print(count)
with open("/Users/priyadalal/Desktop/InputParamComb.json") as example_data:
    data = json.load(example_data)
print(data[count]['layer1']['layer1_name'])
print(data[count]['layer1']['param1'])
print(data[count]['layer1']['param2'])
print(data[count]['layer1']['param3'])

4
Conv2D
30
1
3


In [87]:
# get model configuration params from our json file
# import json 
# with open('Model_Configuration.json') as example_data:
#     data = json.load(example_data)
# print(data['layer1']['layer1_name'])
# int(data['layer1']['param1'])


Now the convolutional NN that we will use to classify the input images taken from the touch screen will have the following layer structure

1. Conv2D
2. MaxPooling2D
3. Flatten
4. Dense
5. Dropout
11. Dense



In [88]:
#layer types and shapes changed in each model

#import tensorflow_model_optimization as tfmot

#quantize_model = tfmot.quantization.keras.quantize_model
model = keras.Sequential()

if (data[count]['layer1']['layer1_name']=='Conv2D'):
    model.add(keras.layers.Conv2D(int(data[count]['layer1']['param1']), kernel_size=(int(data[count]['layer1']['param2']),int(data[count]['layer1']['param3'])), input_shape=input_tensor_shape))
if (data[count]['layer2']['layer2_name']=='Maxpool'):
    model.add(keras.layers.MaxPooling2D(pool_size=(int(data[count]['layer2']['param1']),int(data[count]['layer2']['param2']))))         
model.add(keras.layers.Flatten()) # Flattening the 2D arrays for fully connected layers
model.add(keras.layers.Dense(128, activation=tf.nn.relu))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(10,activation=tf.nn.softmax))


#quantized_model5 = quantize_model(model5)

The Conv2D layer extracts features from the input image using filters that slide across the input image. In this case we will use 28 different filters to extract a large number of unique features that will then be used to classify each image in the later layers. Thus the output of this layer will have the shape (28, 28, 1, 28)

MaxPooling2D is used to reduce the output size of the convolutional layer by reducing each 2 x 2 unique chunk of the output down in to a singular value, this reducing the output's size by a factor of 4. This will reduce our (28, 28, 1, 28) tensor down to a (7, 7, 1, 28) tensor.

The Flatten layer then takes this 2D array (our image) and shapes it into a single dimension (1372).

The following Dense layer reduces the input 1372 values down into 128 classes, taking the first steps in classifying the image into on of the 10 output classes (0-9). This is done using the relu activation function.

The Dropout layer sets 20% of the tensor's values to 0 so as to reduce overfitting.

Finally the last Dense layer reduces the output value down to the 10 classes, each representing a digit between 0 and 9. This is done using the softmax activation function which makes the outputs a set of probabilities summing to 1.


Train the model

In [89]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#quantized_model5.compile(optimizer='adam',
              #loss='sparse_categorical_crossentropy',
              #metrics=['accuracy'])

batch_size = 100
epochs = 10

print("Model 1")
history = model.fit(x=images_train,y=labels_train, epochs=epochs, batch_size=batch_size)
model.summary()




Model 1
Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 13s 221us/sample - loss: 0.6484 - accuracy: 0.7967
Epoch 2/10
60000/60000 [==============================] - 12s 202us/sample - loss: 0.3300 - accuracy: 0.8961
Epoch 3/10
60000/60000 [==============================] - 13s 221us/sample - loss: 0.2734 - accuracy: 0.9125
Epoch 4/10
60000/60000 [==============================] - 11s 184us/sample - loss: 0.2416 - accuracy: 0.9228
Epoch 5/10
60000/60000 [==============================] - 13s 209us/sample - loss: 0.2213 - accuracy: 0.9285
Epoch 6/10
60000/60000 [==============================] - 12s 198us/sample - loss: 0.2084 - accuracy: 0.9334
Epoch 7/10
60000/60000 [==============================] - 17s 283us/sample - loss: 0.1978 - accuracy: 0.9365
Epoch 8/10
60000/60000 [==============================] - 13s 220us/sample - loss: 0.1860 - accuracy: 0.9404
Epoch 9/10
60000/60000 [==============================] - 10s 169us/sample - loss: 0.1795 - accur

Now we can evaluate our trained model using the test data

In [90]:
res = model.evaluate(images_test, labels_test)
print("Model1 has an accuracy of {0:.2f}%".format(res[1] * 100))

# Adding the results to JSON file
stringlist = []
li =[]
model.summary(print_fn=lambda x: stringlist.append(x))
for st in stringlist:
    li.append(st)
    
result={
    'Iteration': str(count),
    'Accuracy is:' : str(res[1] * 100),
     '':str(li[16])
    
}
# with open("Objectives.json", "w") as jsonFile:
#     json.dump(result, jsonFile)



10000/10000 [==============================] - 2s 181us/sample - loss: 0.1557 - accuracy: 0.9475
Model1 has an accuracy of 94.75%


Writing our final results back to our JSON file

In [91]:
#Appending procedure JSON
import os
a = []
if not os.path.isfile("/Users/priyadalal/Desktop/OutputObjective.json"):
    a.append(result)
    with open("/Users/priyadalal/Desktop/OutputObjective.json", mode='w') as f:
        f.write(json.dumps(a, indent=2))
else:
    with open("/Users/priyadalal/Desktop/OutputObjective.json") as feedsjson:
        feeds = json.load(feedsjson)

    feeds.append(result)
    with open("/Users/priyadalal/Desktop/OutputObjective.json", mode='w') as f:
        f.write(json.dumps(feeds, indent=2))

In [92]:
#Updating our counter on completing the Iteration 
count=count+1